In [20]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

load_dotenv()
neo4j_url = os.environ.get("NEO4J_URI")
neo4j_username = os.environ.get("NEO4J_USERNAME")
neo4j_password = os.environ.get("NEO4J_PASSWORD")

# Initialize the Neo4j driver
driver = GraphDatabase.driver(neo4j_url, auth=(neo4j_username, neo4j_password))

In [34]:
import networkx as nx

In [29]:
traj = 7
query_patient_timestep = """
MATCH (p:Patient {traj:  $traj})-[has_timestep]->(ts:TimeStep)
RETURN p, ts
LIMIT 20;
"""

query_actions = """
MATCH (p:Patient {traj: $traj})-[]->(:TimeStep)-[action]->(:TimeStep)
RETURN action
LIMIT 20;
"""

records, summary, keys = driver.execute_query(
    query_patient_timestep, traj=traj,
    
)
print(f'{len(records)=}')
patient = records[0].data()['p']
time_steps = [record.data()['ts'] for record in records]
# for record in records:
#     print(record.data())

records, summary, keys = driver.execute_query(
    query_actions, traj=traj,
    
)
actions = [record["action"]._properties for record in records]

len(records)=7


In [31]:
time_steps

[{'RR': 17.5,
  'paO2': 32.0,
  'Total_bili': 0.2,
  'Calcium': 8.7,
  'PT': 13.4,
  'DiaBP': 71.7,
  'FiO2_1': 0.36,
  'Arterial_pH': 7.36,
  'HR': 82.33333333333333,
  'Glucose': 123.0,
  'Platelets_count': 265.0,
  'INR': 1.2,
  'MeanBP': 97.0,
  'PTT': 26.399999999999995,
  'Chloride': 103.0,
  'BUN': 23.0,
  'Weight_kg': 66.69999694824219,
  'Hb': 11.199999999999998,
  'Magnesium': 2.2,
  'PaO2_FiO2': 88.88888888888889,
  'HCO3': 22.0,
  'SGOT': 13.0,
  'reward': 0.0,
  'Creatinine': 1.2,
  'Temp_C': 37.5,
  'input_4hourly': 0.0,
  'paCO2': 46.0,
  'charttime': 5484989220.0,
  'Arterial_BE': -1.0,
  'SpO2': 96.14285714285715,
  'mechvent': 0.0,
  'cumulated_balance': 0.0,
  'terminal': False,
  'output_4hourly': 0.0,
  'max_dose_vaso': 0.0,
  'SIRS': 0.0,
  'SysBP': 147.6,
  'SOFA': 8.0,
  'input_total': 0.0,
  'Sodium': 136.0,
  'output_total': 0.0,
  'GCS': 15.0,
  'step': 0.0,
  'Arterial_lactate': 2.9,
  'WBC_count': 11.5,
  'Potassium': 5.266666666666667,
  'SGPT': 36.0,
  'S

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GNN_DQN_Agent(nn.Module):
    def __init__(self, input_dim, hidden_dim, action_dim):
        super(DQNGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, action_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        q_values = self.fc(x)
        return q_values

In [ ]:
# Define the environment
class DynamicGraphEnvironment:
    def __init__(self):
        self.graph = nx.Graph()
        G = nx.Graph()
        G.add_node(1)
        G.add_node(2)
        self.step_count = 0
    
    def step(self):
        # Simulate a new node being added to the graph
        new_node = self.graph.number_of_nodes()
        self.graph.add_node(new_node)
        return new_node

# Define the Graph Neural Network (GNN) model
class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, output_num):
        super(GNN, self).__init__()
        self.gcn_layer = nn.GraphConv(input_dim, hidden_dim)
        self.linear_layer = nn.Linear(hidden_dim, output_dim)
        
    
    def forward(self, node_features, edge_index):
        h = self.gcn_layer(x=node_features, edge_index=edge_index)
        h = self.linear_layer(h)
        probabilities = F.softmax(h, dim=-1)
        return probabilities


def adjacency_matrix_to_edge_index(adjacency_matrix):
    # Convert the scipy csr_matrix to a COO (Coordinate) format, which is compatible with torch
    coo_matrix = adjacency_matrix.tocoo()
    # Create a torch tensor for the non-zero elements' indices
    edge_index = torch.tensor([coo_matrix.row, coo_matrix.col], dtype=torch.long)
    # edge_index = torch.nonzero(adjacency_matrix)
    return edge_index.t().contiguous()


# Define the reinforcement learning agent
class RLAgent:
    def __init__(self, state_dim, action_dim, action_num):
        self.state_dim = state_dim
        self.model = GNN(state_dim, HIDDEN_DIM, action_dim, action_num)
        self.action_num = action_num
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
    
    def select_action(self, state):
        
        number_of_nodes = env.graph.number_of_nodes()
        # mock_features = torch.randn(number_of_nodes, self.state_dim)
        mock_features = torch.ones(number_of_nodes, self.state_dim)
        self.x = mock_features
        
        pyg_graph = from_networkx(state)
        
        
        if random.uniform(0, 1) < EPSILON:
            action = np.random.randint(self.action_num) # Explore action space
        
        action_probs = self.model(node_features=self.x, edge_index=pyg_graph.edge_index)
        # action_probs = self.model(node_features=pyg_graph.x, edge_index=pyg_graph.edge_index)
        action = torch.argmax(action_probs)
        
        return action.item()
        # return 0
    
    def update(self, state, action, reward):
        self.optimizer.zero_grad()
        pyg_graph = from_networkx(state)
        action_probs = self.model(node_features=self.x, edge_index=pyg_graph.edge_index)
        loss = -action_probs[action] * reward
        loss.backward()
        self.optimizer.step()
        pass

# Define a reward function
def calculate_reward(node_created, link_created, target_node):
    if link_created:
        return 1.0
    return -1.0  # Penalize if no link created

# Training loop
env = DynamicGraphEnvironment()
# agent = RLAgent(state_dim=env.graph.number_of_nodes(), action_dim=1)
feature_dim = 10
agent = RLAgent(state_dim=feature_dim, action_dim=1, action_num=2)

for episode in range(1000):
    #state is a graph
    state = env.graph
    new_node = env.step()
    
    action = agent.select_action(state)
    print(f"Action {action}")
    link_created = action == new_node
    reward = calculate_reward(new_node, link_created, target_node=new_node)
    
    agent.update(state, action, reward)
    
    print(f"Episode {episode + 1}: Reward {reward}")

In [ ]:
time_variant_properties = ['step', 'm:charttime',
        'o:mechvent', 'o:max_dose_vaso', 'o:re_admission',
       'o:Weight_kg', 'o:GCS', 'o:HR', 'o:SysBP', 'o:MeanBP', 'o:DiaBP',
       'o:RR', 'o:Temp_C', 'o:FiO2_1', 'o:Potassium', 'o:Sodium', 'o:Chloride',
       'o:Glucose', 'o:Magnesium', 'o:Calcium', 'o:Hb', 'o:WBC_count',
       'o:Platelets_count', 'o:PTT', 'o:PT', 'o:Arterial_pH', 'o:paO2',
       'o:paCO2', 'o:Arterial_BE', 'o:HCO3', 'o:Arterial_lactate', 'o:SOFA',
       'o:SIRS', 'o:Shock_Index', 'o:PaO2_FiO2', 'o:cumulated_balance',
       'o:SpO2', 'o:BUN', 'o:Creatinine', 'o:SGOT', 'o:SGPT', 'o:Total_bili',
       'o:INR', 'o:input_total', 'o:input_4hourly', 'o:output_total',
       'o:output_4hourly', 'a:action', 'r:reward']

In [ ]:
def clean_name_of_column(old_name: str):
    if ':' in old_name:
        # Use regex to capture the part after the colon
        match = re.search(r':\s*(.+)', old_name)
        return match.group(1).strip()
    else:
        # If no colon, return the whole text stripped
        return old_name.strip()

In [ ]:
from neo4j import GraphDatabase
import networkx as nx

class Neo4jToNetworkX:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def fetch_graph(self):
        with self.driver.session() as session:
            # Fetch nodes
            nodes_query = """
            MATCH (n)
            RETURN id(n) AS node_id, n.name AS name
            """
            nodes = session.run(nodes_query)

            # Fetch edges
            edges_query = """
            MATCH (n)-[r]->(m)
            RETURN id(n) AS source, id(m) AS target, r.weight AS weight
            """
            edges = session.run(edges_query)

            # Create NetworkX graph
            G = nx.DiGraph()  # Use `` for undirected graphs

            # Add nodes
            for record in nodes:
                G.add_node(record["node_id"], name=record["name"])

            # Add edges
            for record in edges:
                G.add_edge(
                    record["source"], 
                    record["target"], 
                    weight=record.get("weight", 1.0)  # Default weight if none
                )

            return G

# Example usage
converter = Neo4jToNetworkX("bolt://localhost:7687", "neo4j", "password")
graph = converter.fetch_graph()

In [35]:
G = nx.Graph()

G.add_node(patient)

TypeError: unhashable type: 'dict'